# Code Intrepreter

### Create An Assistant With Code Intrepreter

In [1]:
from openai import OpenAI
 
client = OpenAI()

In [61]:
# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("data/revenue-forecast.csv", "rb"),
  purpose='assistants'
)

# Create an assistant using the file ID
assistant = client.beta.assistants.create(
  instructions="You are a personal math tutor. When asked a math question",
  model="gpt-4o",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }
)

### Create A Thread

In [62]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "I need to solve the equation `3x + 11 = 14`. Can you help me?",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

### Create A Run

In [63]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

### Explore Results

In [64]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [67]:
run_steps.dict()["data"]

[{'id': 'step_Pj7nELXwW6xcFDhurAvfYyUx',
  'assistant_id': 'asst_h2xrhA7fkzjezjc5MrxybtS3',
  'cancelled_at': None,
  'completed_at': 1723670039,
  'created_at': 1723670036,
  'expired_at': None,
  'failed_at': None,
  'last_error': None,
  'metadata': None,
  'object': 'thread.run.step',
  'run_id': 'run_znKqCYOdCPiyhQc5mAt5OeCh',
  'status': 'completed',
  'step_details': {'message_creation': {'message_id': 'msg_y51DsxyNad6qr6GPToXSFD3p'},
   'type': 'message_creation'},
  'thread_id': 'thread_Ku1Heyj0ysncwQhUf4gsNkE7',
  'type': 'message_creation',
  'usage': {'completion_tokens': 180,
   'prompt_tokens': 344,
   'total_tokens': 524},
  'expires_at': None}]

In [60]:
print(run_steps.dict()["data"][1]["step_details"]["tool_calls"][0]["code_interpreter"]["input"])

import sympy as sp

# Define the variable
x = sp.symbols('x')

# Define the equation
equation = sp.Eq(3*x + 11, 14)

# Solve the equation
solution = sp.solve(equation, x)
solution
